In [2]:
import csv

In [33]:
with open('TF.GO_enriched2.PMID-jSRE_v2_PD.both.csv') as f:
    reader = csv.DictReader(f)
    data2 = [l for l in reader] 

with open('Supplement2.csv') as f:
    reader = csv.DictReader(f)
    source = [l for l in reader]

In [34]:
src_sents = set(i['sentence'] for i in source)
data2_sents = set(i['Sentence'] for i in data2)

In [35]:
print('src', len(src_sents))
print('data2', len(data2))

src 2228
data2 3516


In [36]:
res = src_sents & data2_sents
len(res)

2223

This looks good, only a few missing. Maybe its b/c of slight differences?

In [37]:
res = set(i.strip() for i in src_sents) & set(i.strip for i in data2_sents)
len(res)

0

It is. So every sentence from out top2500 is in data2. But what about the relations within these sentences?

In [38]:
data2_keys = set((i['Gene_mention1'].lower(), i['Gene_mention2'].lower(), i['Sentence'].strip()) for i in data2)
src_keys = set((i['gene1'].lower(), i['gene2'].lower(), i['sentence'].strip()) for i in source)
print(len(source), len(src_keys))

2500 2499


That should not happen, investigate...

In [39]:
from collections import Counter
Counter((i['gene1'], i['gene2'], i['sentence'].strip()) for i in source).most_common(1)

[(('GATA-binding protein',
   'GATA-4',
   'These results demonstrate the selective role of GATA-4 in the transcriptional regulation of the IL-5 gene in a circumstance where multiple members of the GATA-binding proteins are expressed'),
  2)]

This entry is part of two different abstracts (different pubmed id). data2 contains the entry 2 times as well with the same prediction. so ignore for now.

In [40]:
c = 0
missing = []
d = data2_keys.copy()
for g1, g2, t in src_keys:  # ignore order of genes
    if (g1, g2, t) in d:
        c += 1
        d.remove((g1,g2,t))
    elif (g2, g1, t) in d:
        c += 1
        d.remove((g2, g1, t))
    else:
        missing.append((g1, g2, t))
c

2445

still 55 missing due to gene name normalisation issues as shown below:

In [42]:
missing[6]

('coup-tf',
 'rarbeta',
 'In a transient transfection assay, COUP-TF strongly induced transcriptional activity of the RARbeta promoter in a RA- and RARalpha-dependent manner.')

In [43]:
[d for d in data2_keys if d[2].startswith('In a transient transfection assay, COUP-TF strongly ')]

[('raralpha',
  'coup-tf',
  'In a transient transfection assay, COUP-TF strongly induced transcriptional activity of the RARbeta promoter in a RA- and RARalpha-dependent manner.'),
 ('rarbeta',
  'coup-tf',
  'In a transient transfection assay, COUP-TF strongly induced transcriptional activity of the RARbeta promoter in a RA- and RARalpha-dependent manner.')]

## Conclusion

- every interaction described in Supplements2.xls is also available in TF.GO_enriched2.PMID-jSRE_v2_PD.both.csv
- gene1 / gene2 might be switched
- gene names are not normalized the same way